In [1]:
suppressPackageStartupMessages({
    library(sf)
    library(ggthemes)
    library(purrr)
    library(glue)
    library(furrr)
    library(sfarrow)
    library(future)
    library(dplyr)
    library(data.table)
    library(ggplot2)
    library(igraph)
    library(Rcpp)
})


fig.size <- function(h, w) {
    options(repr.plot.height = h, repr.plot.width = w)
}

In [2]:
library(devtools)

In [52]:
# devtools::load_all('..')

In [1]:
devtools::document('..')

ℹ Updating spatula documentation

ℹ Loading spatula

Loading required package: Rcpp

Loading required package: RcppArmadillo



Writing NAMESPACE


Warning message:
“infer_polygons.Rd is missing name/title. Skipping”


Writing NAMESPACE


In [2]:
## git add . --all
## git commit -m '...'
devtools::install_git('..', dependencies = FALSE)

✔  checking for file ‘/tmp/RtmpsLAyE5/file25a5b249e004c6/DESCRIPTION’
─  preparing ‘spatula’:
✔  checking DESCRIPTION meta-information
─  cleaning src
─  checking for LF line-endings in source and make files and shell scripts
─  checking for empty or unneeded directories
─  building ‘spatula_0.0.0.9000.tar.gz’
   


Installing package into ‘/mnt/rstudio/ilya/R/x86_64-pc-linux-gnu-library/4.1’
(as ‘lib’ is unspecified)



In [2]:
library(spatula)


Loading required package: RcppArmadillo



# Data

In [3]:
load('../data/spatula_cosmx.rda')

In [4]:
head(tx)
nrow(tx)

gene,cell,x,y
<chr>,<int>,<dbl>,<dbl>
KRT86,0,357.8340,232.1760
KRT86,2642,499.2180,222.3600
KRT86,1024,498.4053,315.8624
KRT86,0,379.0131,365.4386
KRT86,1097,446.7708,294.9840
KRT86,1343,424.6807,224.5995


[1] 153214

In [5]:
data.frame(head(cells))
nrow(cells)

,id,shape,neighborhood,ntx_cell,ngene_cell,ntx_neighbors,ngene_neighbors
,<chr>,<POLYGON>,<POLYGON>,<dbl>,<int>,<dbl>,<int>
1,920,POLYGON ((492.0364 347.7475...,POLYGON ((469.1623 366.1447...,539,198,6814,726
2,923,POLYGON ((362.5765 349.5369...,POLYGON ((339.2242 352.7786...,236,128,4567,682
3,935,POLYGON ((409.3373 349.9752...,"POLYGON ((384.419 356.0701,...",476,187,6660,759
4,946,"POLYGON ((520.715 338.3761,...",POLYGON ((500.3057 334.5539...,516,183,7683,737
5,953,POLYGON ((497.6366 348.2479...,POLYGON ((473.1845 337.8278...,233,114,7025,720
6,956,"POLYGON ((414.061 334.1596,...","POLYGON ((406.544 315.5595,...",800,283,7990,771


[1] 202

# Run it 

## 1

In [10]:
output_dir='tmp'
fname_baysor_config='config.toml'
# baysor_binary='/n/data1/bwh/medicine/korsunsky/lab/rom4535/software/baysor'
opts = list(
    baysor_mode = 'docker', ## baysor can be run through docker container or natively installed binary 
    max_tx_per_grid = 10000, ## maximum number of transcripts per tile 
    compute_shapes = TRUE, 
    max_parallel_jobs = 60, ## maximum number of concurrent Baysor jobs running 
    min_tx_per_cell = 10, 
    remove_temp_files = TRUE ## set to FALSE for debugging 
)



In [11]:
ntiles <- baysor.split_tx_files(output_dir, opts$max_tx_per_grid) 
baysor.prepare_cmds(ntiles, output_dir, fname_baysor_config, opts$max_parallel_jobs)
baysor.run_baysor_tiles_docker(ntiles, output_dir, fname_baysor_config, max_parallel_jobs=opts$max_parallel_jobs) 
tx <- baysor.collect_tx(output_dir, testing=FALSE, mintx=opts$min_tx_per_cell) 
counts <- tx_to_counts(tx$gene, tx$cell, remove_bg = TRUE)
cells <- baysor.collect_cells(tx, opts$min_tx_per_cell)
environment(baysor.finish) <- environment() 
baysor.finish() 

In [13]:
list.files('tmp')

st_read_parquet('tmp/shapes.parquet') %>% head(5) %>% data.frame()
fread('tmp/cells.csv', nrows = 5)
spatula::readMM('tmp/counts.mtx')[1:5, 1:5]
readLines('tmp/genes.tsv', n = 5)
fread('tmp/transcripts.csv', nrows = 5)

[1] "cells.csv"       "counts.mtx"      "genes.tsv"       "shapes.parquet" 
[5] "transcripts.csv"

,id,shape
,<chr>,<POLYGON>
1,1,"POLYGON ((359.0275 171.204,..."
2,2,"POLYGON ((366.345 178.4798,..."
3,3,POLYGON ((378.3308 171.1735...
4,4,POLYGON ((370.1091 196.1505...
5,5,POLYGON ((388.2981 171.2572...


id,x,y,area,n_transcripts,avg_confidence,cluster,rand_color
<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<int>,<chr>
1,356.1528,178.2853,120.7991,751,0.8694541,4,#bcbd22
2,366.9136,183.7355,104.8429,359,0.8366574,1,#dbdb8d
3,378.7970,179.2105,135.3296,813,0.8245018,4,#2ca02c
4,363.6378,193.7154,126.9589,528,0.8092045,4,#f7b6d2
5,391.8998,175.7672,106.9247,697,0.8774462,4,#7f7f7f


5 x 5 sparse Matrix of class "dgCMatrix"
              
[1,] . . . . .
[2,] . . . . .
[3,] . . . . 2
[4,] . . . . .
[5,] . 1 . . .

[1] "AATK" "ABL1" "ABL2" "ACE"  "ACE2"

gene,cell,x,y,molecule_id,prior_segmentation,confidence,cluster,assignment_confidence,is_noise,ncv_color,N
<chr>,<int>,<dbl>,<dbl>,<int>,<int>,<dbl>,<int>,<dbl>,<lgl>,<chr>,<int>
KRT86,9,381.2092,197.3803,1,2689,0.99996,2,0.98,FALSE,#00B7D3,204
KRT86,16,388.4400,219.5220,2,1366,1.00000,2,1.00,FALSE,#009BD9,225
KRT86,14,378.7273,216.1332,3,1361,0.99998,4,0.90,FALSE,#002F45,237
KRT86,16,389.6978,215.4735,4,1366,1.00000,2,0.90,FALSE,#0083AA,225
BECN1,5,386.7510,177.4740,5,0,1.00000,4,0.88,FALSE,#730E7E,697


## full

In [16]:
baysor.run <- function(
    tx, 
    fname_baysor_config, 
    output_dir, 
    opts = list(
        baysor_mode = c('docker', 'binary')[1], ## baysor can be run through docker container or natively installed binary 
        max_tx_per_grid = 2e7, ## maximum number of transcripts per tile 
        compute_shapes = TRUE, 
        max_parallel_jobs = 10, ## maximum number of concurrent Baysor jobs running 
        min_tx_per_cell = 10, 
        remove_temp_files = TRUE ## set to FALSE for debugging 
    )
) {
    ## Check inputs 
    message('WARNING: for now, tx needs to have columns: x, y, gene, cell')
    stopifnot(colnames(tx) == c('x', 'y', 'gene', 'cell'))
    stopifnot(file.exists(fname_baysor_config))
    if (baysor_mode == 'docker') {
        if (!'docker' %in% strsplit(system('groups', intern = TRUE), ' ')[[1]]) {
            stop('You do not belong to the docker group. Please follow the instructions here to run docker without root privileges: https://docs.docker.com/engine/install/linux-postinstall/')
        }
    }
    
    ntiles <- baysor.split_tx_files(output_dir, opts$max_tx_per_grid) 
    baysor.prepare_cmds(ntiles, output_dir, fname_baysor_config, opts$max_parallel_jobs)
    if (baysor_mode == 'docker') {
        baysor.run_baysor_tiles_docker(ntiles, output_dir, fname_baysor_config)    
    } else if (baysor_mode == 'binary') {
        stop('binary mode not implemented yet. Please use docker mode.')
        # baysor.run_baysor_tiles_binary(ntiles, output_dir, ..., fname_baysor_config) 
    } else {
        stop('invalid baysor_mode')
    }
    
    tx <- baysor.collect_tx(output_dir, testing=FALSE, mintx=opts$min_tx_per_cell) 
    counts <- tx_to_counts(tx$gene, tx$cell, remove_bg = TRUE)
    cells <- baysor.collect_cells(tx, opts$min_tx_per_cell)
    environment(baysor.finish) <- environment() 
    baysor.finish() 
}
